In [1]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

2025-06-30 21:51:53.889801: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
data = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv",
    sep=";",
)
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [7]:
# Split the data into Training, validation and test sets

train, test = train_test_split(data, test_size=0.25, random_state=42)

In [10]:
train_x = train.drop(['quality'], axis=1).values
train_y = train[['quality']].values.ravel()

# Validation dataset
test_x = test.drop(['quality'], axis=1).values
test_y = test[['quality']].values.ravel()

train_x, valid_x, train_y, valid_y = train_test_split(
    train_x, train_y, test_size=0.20, random_state=42
)

signature = infer_signature(train_x, train_y)

In [11]:
# ANN Model

def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):

    #Define model architecture
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)

    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1)
        ]
    )

    # Compile the Model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"], momentum=params["momentum"]
    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    # Train the ANN model with lr and momentum params with MLFlow tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x, train_y, validation_data=(valid_x, valid_y),
                  epochs=epochs,
                  batch_size=64)
        
        # Evaluate the model
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)

        eval_rmse = eval_result[1]

        # Log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        # Log the model
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}



In [12]:
def objective(params):
    #MLFlow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x = train_x,
        train_y = train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

In [13]:
space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momenturm", 0.0, 1.0)
}

In [15]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    # Conduct the Hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval runs: {best_run['loss']}")


Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1:03 1s/step - loss: 35.7624 - root_mean_squared_error: 5.9802
17/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 24.2225 - root_mean_squared_error: 4.8832 
27/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 20.0254 - root_mean_squared_error: 4.4026
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 15.3587 - root_mean_squared_error: 3.8023
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 15.1878 - root_mean_squared_error: 3.7790 - val_loss: 1.9077 - val_root_mean_squared_error: 1.3812

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - loss: 2.7157 - root_mean_squared_error: 1.6479
18/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.8136 - root_mean_squared_error: 1.3431 
32/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.7244 - root_mean_squared_error: 1.3106
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.6759 - root_mean_squared_error: 1.2925 - val_loss: 1.4939 - val_root_

2025/06/30 22:34:53 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                                                     

 1/46 ━━━━━━━━━━━━━━━━━━━━ 34s 776ms/step - loss: 34.1393 - root_mean_squared_error: 5.8429
14/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 32.5827 - root_mean_squared_error: 5.7080   
20/46 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 32.3591 - root_mean_squared_error: 5.6883
21/46 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 32.3174 - root_mean_squared_error: 5.6846
22/46 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 32.2726 - root_mean_squared_error: 5.6807
26/46 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 32.0859 - root_mean_squared_error: 5.6641
30/46 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 31.9072 - root_mean_squared_error: 5.6482
33/46 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 31.7791 - root_mean_squared_error: 5.6368
34/46 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 31.7343 - root_mean_squared_error: 5.6328
38/46 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 31.5636 - root_mean_squared_error: 5.6175
44/46 ━━━━━━━━━━

2025/06/30 22:35:17 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                                                     

 1/46 ━━━━━━━━━━━━━━━━━━━━ 32s 730ms/step - loss: 33.5201 - root_mean_squared_error: 5.7897
18/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 10.8925 - root_mean_squared_error: 3.1572   
32/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.8219 - root_mean_squared_error: 2.6356 
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 6.2154 - root_mean_squared_error: 2.3270 - val_loss: 0.8899 - val_root_mean_squared_error: 0.9433

Epoch 2/3                                                                     

 1/46 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - loss: 0.6252 - root_mean_squared_error: 0.7907
17/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8455 - root_mean_squared_error: 0.9187 
22/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.8365 - root_mean_squared_error: 0.9139
34/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.8157 - root_mean_squared_error: 0.9026
45/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.7969 - root_mean_

...

Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1:04 1s/step - loss: 42.0617 - root_mean_squared_error: 6.4855
 8/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 17.7438 - root_mean_squared_error: 4.0565 
13/46 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 13.4265 - root_mean_squared_error: 3.4769
16/46 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 11.8606 - root_mean_squared_error: 3.2473
19/46 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 10.6871 - root_mean_squared_error: 3.0668
20/46 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 10.3565 - root_mean_squared_error: 3.0144
25/46 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 9.0177 - root_mean_squared_error: 2.7942 
27/46 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 8.5935 - root_mean_squared_error: 2.7215
29/46 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 8.2164 - root_mean_squared_error: 2.6558
38/46 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 6.9300 - root_mean_squared_error: 2.4219
46/46 ━━━━━━━━━━━━━━━━━

2025/06/30 22:35:49 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



100%|██████████| 4/4 [01:15<00:00, 18.88s/trial, best loss: 0.7230923771858215]

2025/06/30 22:36:05 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Best parameters: {'lr': 0.06421584021139144, 'momenturm': 0.30345961635761254}
Best eval runs: 0.7230923771858215
